In [11]:
import json
import numpy as np
import re
from grl.p2sro.payoff_table import PayoffTable

# psro_payoff_table_checkpoint_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_08.41.48PM_Dec-28-2020/payoff_table_checkpoints/payoff_table_checkpoint_25.json"
# psro_policy_nums_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_08.41.48PM_Dec-28-2020/payoff_table_checkpoints/policy_nums_checkpoint_25.json"

# psro_payoff_table_checkpoint_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_10.59.03PM_Dec-28-2020/payoff_table_checkpoints/payoff_table_checkpoint_30.json"
# psro_policy_nums_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_10.59.03PM_Dec-28-2020/payoff_table_checkpoints/policy_nums_checkpoint_30.json"

psro_payoff_table_checkpoint_path = "/home/jb/git/grl/grl/data/kuhn_psro/manager_06.02.17PM_Jan-14-2021/payoff_table_checkpoints/payoff_table_checkpoint_19.json"
psro_policy_nums_path = "/home/jb/git/grl/grl/data/kuhn_psro/manager_06.02.17PM_Jan-14-2021/payoff_table_checkpoints/policy_nums_checkpoint_19.json"

original_date = re.search("\d\d\.\d\d.\d\d.\w\w\w\w\w\W\d\d\W\d\d\d\d", psro_payoff_table_checkpoint_path)[0]
poker_version = "leduc_poker"

with open(psro_policy_nums_path, "r") as policy_nums_file:
    policy_nums = json.load(policy_nums_file)
largest_fixed_policy_nums = (max(policy_nums['0']['fixed_policies']), max(policy_nums['1']['fixed_policies']))

payoff_table = PayoffTable.from_json_file(json_file_path=psro_payoff_table_checkpoint_path)

In [12]:
from grl.rl_apps.kuhn_poker_p2sro.poker_utils import get_stats_for_single_payoff_table
from grl.rl_apps.kuhn_poker_p2sro.poker_multi_agent_env import PokerMultiAgentEnv
from grl.rl_apps.kuhn_poker_p2sro.config import kuhn_sac_params, kuhn_dqn_params, leduc_dqn_params
from ray.rllib.agents.dqn import SimpleQTorchPolicy, SIMPLE_Q_DEFAULT_CONFIG
from ray.rllib.utils import merge_dicts

env_config = {'version': poker_version, "fixed_players": True, "append_valid_actions_mask_to_obs": True}
env = PokerMultiAgentEnv(env_config=env_config)
num_players = 2

stats = get_stats_for_single_payoff_table(
    payoff_table=payoff_table,
    highest_policy_num=min(largest_fixed_policy_nums),
    # highest_policy_num=6,
    poker_env_config=env_config,
    policy_class=SimpleQTorchPolicy,
    policy_config=merge_dicts(SIMPLE_Q_DEFAULT_CONFIG, leduc_dqn_params(action_space=env.action_space))
)


2021-01-14 20:24:38,847	ERROR worker.py:660 -- Calling ray.init() again after it has already been called.
2021-01-14 20:24:38,851	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.leduc_dqn.valid_actions_fcnet.LeducDQNFullyConnectedNetwork'> as None
2021-01-14 20:24:38,855	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.leduc_dqn.valid_actions_fcnet.LeducDQNFullyConnectedNetwork'> as None
2021-01-14 20:24:38,860	INFO torch_policy.py:107 -- TorchPolicy running on GPU.
2021-01-14 20:24:38,869	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.leduc_dqn.valid_actions_fcnet.LeducDQNFullyConnectedNetwork'> as None
2021-01-14 20:24:38,872	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.leduc_dqn.valid_actions_fcnet.LeducDQNFullyConnectedNetwork'> as None
2021-01-14 20:24:38,876	INFO torch_policy.py:107 -- TorchPolicy running on GPU.


payoff matrix as 0 (row) against 1 (columns):
[[-1.35880005]]
payoff matrix as 1 (row) against 0 (columns):
[[1.35880005]]
payoff matrix as 1 (row) against 0 (columns):
[[1.35880005]]
1 policies, 3.3416666666666672 exploitability
payoff matrix as 0 (row) against 1 (columns):
[[-1.35880005 -1.83959997]
 [ 1.93330002 -1.23319995]]
payoff matrix as 1 (row) against 0 (columns):
[[ 1.35880005 -1.93330002]
 [ 1.83959997  1.23319995]]
payoff matrix as 1 (row) against 0 (columns):
[[ 1.35880005 -1.93330002]
 [ 1.83959997  1.23319995]]
2 policies, 3.2328626090656782 exploitability
payoff matrix as 0 (row) against 1 (columns):
[[-1.35880005 -1.83959997  0.075     ]
 [ 1.93330002 -1.23319995 -4.16610003]
 [ 0.60804999  0.90764999 -0.7579    ]]
payoff matrix as 1 (row) against 0 (columns):
[[ 1.35880005 -1.93330002 -0.60804999]
 [ 1.83959997  1.23319995 -0.90764999]
 [-0.075       4.16610003  0.7579    ]]
payoff matrix as 1 (row) against 0 (columns):
[[ 1.35880005 -1.93330002 -0.60804999]
 [ 1.839

In [15]:
stats

{'num_policies': [1, 2, 3, 4, 5],
 'exploitability': [3.3416666666666672,
  3.2328626090656782,
  3.0383947432481033,
  2.4662029411257715,
  3.178387273692497],
 'timesteps': [643968, 1287936, 1995904, 2639872, 3283840],
 'episodes': [191341, 318026, 458593, 581061, 708462]}

In [16]:

import itertools
import pandas as pd
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = 'png'
pio.renderers.default = "browser"

df = pd.DataFrame.from_dict(data=stats)
fig = px.line(df, x="episodes", y="exploitability",
              title='Leduc Poker PSRO Exploitability',
              # range_x=[0, int(3e6)],
              )
fig.show()

In [ ]:
save_path = f"/home/jb/git/grl/grl/data/psro_stats_{poker_version}_{original_date}.json"
with open(save_path, "+w") as json_file:
    json.dump(stats, json_file)
print(f"saved to {save_path}")